workflow for reading adsorbate structures (of hydrocarbons) on metal surfaces and determining their effective site ensembles in catalysis. 

In [1]:
import pandas as pd
import numpy as np
import scipy as sci
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import copy

from ase import Atoms

from ase.visualize import view
from ase.io import read
from ase.io import write
from ase.build import molecule
from ase import units
import os

In [2]:
import sys

path_custom_fns = "/custom_functions"
sys.path.append(path_custom_fns)

from excluded_area import (center_ads, build_ads_grid, build_atom_volume,
                           mol_2_vol_area, mol_overlap_2D)

Import adsorbate geometry file using Atomic Simulation Environment

In [ ]:
#input path to geometry file
name_file = 'CO_FF_opt_example.xyz'

#use ASE to read file and open as atoms object
atoms = read(name_file)

#grid cube dimension
d = 0.04 #discrete size

#build datafrom from structure coordinates
df_atoms = pd.DataFrame(list(atoms.symbols),columns=['atom'])
df_pos = pd.DataFrame(atoms.get_positions(),columns=['x','y','z'])
df = pd.concat([df_atoms,df_pos], axis=1)

#grid excess to ensure grid extends beyond vdW radii
grid_ex = 3

In [ ]:
#build adsorbate from the geometry file
M_H, M_C, M_O, M_ads, COP = center_ads(atoms)

#build 3D grid to enclose the adsorbate
M_size,M_fill = build_ads_grid(M_ads,d,grid_ex)

In [ ]:
#obtain the grid positions of the C and H atoms in the M matrix
pos_H = M_H/d+np.ceil(M_size/2)
pos_C = M_C/d+np.ceil(M_size/2)
pos_O = M_O/d+np.ceil(M_size/2)

In [ ]:
#specify the van der Waals radii of C, H, and O
vdW_R_C = 1.7
vdW_R_H = 1.2
vdW_R_O = 1.52

# get matrices to represent filled spheres for each element
C_size, C_fill = build_atom_volume(vdW_R_C,d)
H_size, H_fill = build_atom_volume(vdW_R_H,d)
O_size, O_fill = build_atom_volume(vdW_R_O,d)

In [ ]:
#calculate vdW area by adding in each element
M_fill,vdW_area = mol_2_vol_area(M_fill,pos_C,C_size,C_fill,d)
M_fill,vdW_area = mol_2_vol_area(M_fill,pos_H,H_size,H_fill,d)
M_fill,vdW_area = mol_2_vol_area(M_fill,pos_O,O_size,O_fill,d)

In [ ]:
#find the radius of a circle with equivalent area to the molecules' vdW area
R_ads_eff = np.sqrt(vdW_area/np.pi)

#define a probe to trace the adsorbate for enclosed area calculation
#use the original adsorbate as the probe to represent a single-component adlayer
R_probe = R_ads_eff

#build a matrix to represent the probe volume
pr_size, pr_fill = build_atom_volume(R_probe,d)

#build a 2D matrix representing the areal projection of the probe
pr_fill_2d = np.sum(pr_fill,axis=2)
pr_fill_2d[pr_fill_2d>1] = 1

In [ ]:
#initialize a matrix to represent adsorbate in a larger grid to accomodate a tracing probe

#start with the 2D projection of the adsorbate's volume
M_fill_2d = np.sum(M_fill,axis=2)
M_fill_2d[M_fill_2d>1] = 1

#re-size grid the grid to extent beyond the limits of the adsorbate to accomodate the probe tracer
#make a grid with dimension that matches diameter of adsorbate + 2x probe * 1.3
l_grid = (R_ads_eff*2+2*(R_probe*2))*1.3

#initialize the grid to use for probe-trace calculation
M_fill_probe = np.zeros([int(np.ceil(l_grid/d)),int(np.ceil(l_grid/d))])

x1,y1 = np.shape(M_fill_probe)
x2,y2 = np.shape(M_fill_2d)

#the location in the center
x3 = int(np.floor((x1-x2)/2))
y3 = int(np.floor((y1-y2)/2))

#place the adsorbate at the origin of the M_fill_probe matrix
M_fill_probe[x3:x3+x2,y3:y3+y2] = M_fill_2d

In [ ]:
#initialize a grid to trace the molecular volume with the probe. Has the same dimensions as M_fill_probe 

x_size = np.shape(M_fill_probe)[0]
y_size = np.shape(M_fill_probe)[1]

probe_grid = np.zeros([x_size,y_size])

#loop over the probe_grid matrix and test whether the probe overlaps with the molecular volume in the M_fill_probe matrix
for i in range(int(x_size-pr_size[0])):
    for j in range(int(y_size-pr_size[1])):
        overlap = 0
        #assign probe's position
        pos_probe = np.array([i+pr_size[0]/2,j+pr_size[1]/2])
        #determine if probe overlaps with adsorbate
        overlap = mol_overlap_2D(M_fill_probe,pos_probe,pr_size[0:2],pr_fill_2d,d)
        #identify overlap at sample position
        probe_grid[int(i+pr_size[0]/2),int(j+pr_size[1]/2)]=overlap


In [ ]:
#calculate probe traced area
area_probe = np.sum(probe_grid)*d**2

#convert area trace to excluded area:
R_trace = np.sqrt(area_probe/np.pi)
R_eff = R_trace-R_probe
A_eff = np.pi*R_eff**2

#export excluded area
f = open("ads_probe_enclosed.txt", "a")
f.write(str(A_eff))
f.close()

In [ ]:
print(vdW_area)
print(A_eff)